In [1]:
%cd ..

/hdd/aouadt/these/projets/3d_segm


In [2]:
from importlib import reload
import itertools
import random
from functools import reduce
from os.path import join
from time import time
import pathlib
import os
import re
from IPython.display import display


from tqdm import tqdm
import pandas as pd
import cv2
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data.dataloader import DataLoader
import pytorch_lightning as pl
import skimage.morphology as morp
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from PIL import Image, ImageDraw
import torchviz
# from deep_morpho.datasets.mnist_dataset import MnistClassifDataset
# from deep_morpho.models import LightningBiMoNNClassifier, BiSE, COBiSE, BiSEC, COBiSEC


import deep_morpho.models.softplus as sfp
import general.array_morphology as arm
import general.structuring_elements as selm
from general.nn.loss import DiceLoss
import deep_morpho.threshold_fn as threshold_fn
import deep_morpho.morp_operations as morpop
import deep_morpho.models.threshold_layer as threshold_layer
import deep_morpho.models.complementation_layer as complementation_layer
import deep_morpho.models.pconv as pconv
import deep_morpho.models.lightning_pconv as lpconv
import deep_morpho.models.bise as bise
import deep_morpho.models.lui as lui
import deep_morpho.models.bisel as bisel
import deep_morpho.models.cobise as cobise
import deep_morpho.models.dilation_sum_layer as dis
import deep_morpho.models.bimonn as bimonn
import deep_morpho.models as models
import deep_morpho.models.lightning_bise as lbise
import deep_morpho.models.lightning_cobise as lcobise
import deep_morpho.models.lightning_bimonn as lbimonn
import deep_morpho.datasets.generate_forms1 as gfo
import deep_morpho.datasets.generate_forms2 as gfo2
import deep_morpho.datasets.generate_forms3 as gfo3
import deep_morpho.datasets.axspa_roi_dataset as axd
import deep_morpho.datasets.multi_rect_dataset as mrda
import deep_morpho.datasets.mnist_dataset as mnist_dataset
import deep_morpho.observables.plot_parameters as obs_weights
import deep_morpho.observables.plot_pred as obs_pred
import deep_morpho.observables.weight_histogram as weight_histogram
import deep_morpho.observables as obs
import general.nn.viz.plot_histogram as phist
import general.nn.viz.element_image as eltimage
import general.nn.viz.element_histogram as elthistogram
import deep_morpho.viz.morp_operations_viz as mov
import deep_morpho.viz.elt_generator_bimonn_forward_save as eltgenbifor
import deep_morpho.viz.elt_generator_bimonn_histogram as eltgenbihist
import deep_morpho.viz.bimonn_viz as bimonn_viz
import deep_morpho.save_results_template.load_args as largs
import deep_morpho.save_results_template.display_results as dr

def reload_modules():
    for modl in [sfp, arm, selm, threshold_fn, morpop,
                 threshold_layer,complementation_layer, pconv, lpconv, bise, lui, bisel, cobise,
                 dis, bimonn, models, lbise, lbimonn, lcobise, gfo, gfo2, gfo3, axd,
                 mrda, mnist_dataset, obs_weights, obs_pred, obs,
                weight_histogram, phist, eltimage, elthistogram, mov, eltgenbifor, eltgenbihist, bimonn_viz, largs, dr]:
        reload(modl)
        
reload_modules()

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
print(device)

cuda


# Individual Experiments

In [ ]:
reload_modules()
# tb_path = ("deep_morpho/results/DGMM_2022/sandbox/1/axspa_roi/version_7")
# tb_path = "/hdd/aouadt/these/projets/3d_segm/deep_morpho/results/results_tensorboards/Bimonn_exp_56/sandbox/2/softplus/diskorect/opening/disk/version_1"
# tb_path = "deep_morpho/results/results_tensorboards/Bimonn_exp_59/sandbox/0/softplus/diskorect/translation_erosion/hstick/version_0"
tb_path = "deep_morpho/results/results_tensorboards/Bimonn_exp_59/sandbox/0/softplus/diskorect/translation_erosion/dcross/version_0"

tb_path = join(tb_path, 'checkpoints', os.listdir(join(tb_path, 'checkpoints'))[0])

model = models.LightningBiMoNN.load_from_checkpoint(tb_path)
model.to(device)

loss = nn.BCELoss()

# Multi Experiments comparison Bise

In [3]:
def list_dir_joined(folder: str):
    return [os.path.join(folder, k) for k in os.listdir(folder)]

path_global = "deep_morpho/results/results_tensorboards/Bimonn_mega_multi_1/softplus/diskorect/"
all_paths = []
for operation in os.listdir(path_global):
    if os.path.isdir(join(path_global, operation)):
        for selem in os.listdir(join(path_global, operation)):
            all_paths += sorted(list_dir_joined(join(path_global, operation, selem)), key=lambda x: int(
                re.findall(r'version_(\d+)$', x)[0]
            ))[:-1]

In [4]:
reload_modules()
results = dr.DisplayResults().get_all_results_from_tensorboard(all_paths, load_long_args=False)

for r in results:
    r.update(r['args'])

Reading results: 100%|██████████| 954/954 [00:37<00:00, 25.16it/s]


In [5]:
df = pd.DataFrame(results)
df['dice'] = df['dice'].astype(float)
df['binary_mode_dice'] = df['binary_mode_dice'].astype(float)
df['operation'] = df['experiment_subname'].apply(lambda x: pathlib.Path(x).parent.stem)
df['selem'] = df['experiment_subname'].apply(lambda x: pathlib.Path(x).stem)

In [6]:
tb_path = df[
    (df['learning_rate'] == '0.01') &
    (df['loss_data_str'] == 'MaskedDiceLoss') &
    (df['init_weight_mode'] == 'CUSTOM_CONSTANT') &
    (df['bias_optim_mode'] == 'POSITIVE_INTERVAL_REPARAMETRIZED') &
    (df['operation'] == 'dilation') & 
    (df['selem'] == 'disk')
].tb_path
# with open(join(tb_path, 'seed.txt'), 'r') as f:
#     s = f.read()
# s

In [14]:
tb_path.iloc[0]

'deep_morpho/results/results_tensorboards/Bimonn_mega_multi_1/softplus/diskorect/dilation/disk/version_50'

#### Binary Mode Success Number

In [7]:
changing = ["operation", "selem", ]
metrics = ['dice', 'binary_mode_dice']

df.groupby(changing)[metrics].mean()

dice  binary_mode_dice
operation    selem                             
black_tophat dcross  0.456932          0.486920
             disk    0.468572          0.482921
             hstick  0.339391          0.335720
closing      dcross  0.939893          0.901833
             disk    0.968185          0.867027
             hstick  0.975015          0.933311
dilation     dcross  0.997769          0.997736
             disk    0.996638          0.996191
             hstick  0.999956          1.000000
erosion      dcross  0.592276          0.613526
             disk    0.575092          0.620956
             hstick  0.705869          0.723463
opening      dcross  0.654607          0.640022
             disk    0.756457          0.626815
             hstick  0.940413          0.824720
white_tophat dcross  0.392134          0.456865
             disk    0.382384          0.372300
             hstick  0.221990          0.283608

In [8]:
pd.DataFrame(df.loc[df['binary_mode_dice'] == 1, ["operation", "selem"]].value_counts().sort_values()).sort_values(by=['operation', 'selem'])

0
operation    selem     
black_tophat dcross  14
             disk     9
             hstick   5
closing      dcross  29
             disk    14
             hstick  17
dilation     dcross  49
             disk    49
             hstick  53
erosion      dcross  27
             disk    19
             hstick  37
opening      dcross  14
             disk     4
             hstick  17
white_tophat dcross   1

In [23]:
display(pd.DataFrame(df.loc[(df['dice'] == 1) & (df['init_weight_mode'] == "CUSTOM_CONSTANT"), ["operation", "selem"]].value_counts().sort_values()).sort_values(by=['operation', 'selem']))
display(pd.DataFrame(df.loc[(df['dice'] == 1), ["operation", "selem"]].value_counts().sort_values()).sort_values(by=['operation', 'selem']))


0
operation    selem     
black_tophat dcross   6
             disk     3
             hstick   3
closing      dcross  11
             disk     1
             hstick   6
dilation     dcross  17
             disk    16
             hstick  15
erosion      dcross  10
             disk     8
             hstick  11
opening      dcross   2
             disk     1
             hstick   5
white_tophat dcross   1

0
operation    selem     
black_tophat dcross  18
             disk    11
             hstick   9
closing      dcross  25
             disk     9
             hstick  15
dilation     dcross  48
             disk    48
             hstick  49
erosion      dcross  17
             disk    16
             hstick  20
opening      dcross   3
             disk     3
             hstick  17
white_tophat dcross   2
             hstick   1

#### Best Set of hyperparameters

In [7]:
changing = [
    'loss_data_str',
    'bias_optim_mode',
    'learning_rate',
    'init_weight_mode'
]
metrics = ['dice', 'binary_mode_dice']

results = {}
sums = {}
sums_style = {}


for L in range(1, len(changing) + 1):
    for subset in itertools.combinations(changing, L):
        df_summary = df.pivot_table(index=subset, columns=['operation', 'selem'], values='binary_mode_dice', aggfunc=lambda x: (x==1).any())#.astype(int)
        sums[subset] = pd.DataFrame(df_summary.sum(axis=1) / len(df_summary.columns))
        sums_style[subset] = sums[subset].style.format("{:,.0%}".format)
        results[subset] = df_summary




In [26]:
len(df_summary.columns) * len(df_summary)

954

In [19]:
df_all = df.pivot_table(index=changing, columns=['operation', 'selem'], values=['binary_mode_dice', 'dice'], aggfunc='mean').rename(columns={'dcross': 'dc', 'disk': 'di', 'hstick': 'hs'})
df_all.style.background_gradient(cmap='RdBu', vmin=0, vmax=1).format('{:.2f}')


In [18]:
def color_results(val):
    if val == True:
        return f'color: green'
    elif val == False:
        return f'color: red'
    elif val is None:
        return f'color: blue'

def display_color(df):
    display(df.style.applymap(color_results))

# results[tuple(changing)].style.applymap(color_results)
display_color(results[tuple(changing)])
# display_color(results[("loss_data_str", "init_weight_mode")])

In [12]:
pd.DataFrame(results[tuple(changing)].sum(1) / len(results[tuple(changing)].columns)).style.format('{:.0%}')

# Bimonn Multi Sybisel

In [3]:
def list_dir_joined(folder: str):
    return [os.path.join(folder, k) for k in os.listdir(folder)]

path_global = "/hdd/aouadt/these/projets/3d_segm/deep_morpho/results/results_tensorboards/Bimonn_exp_63/multi/sybisel/softplus/diskorect/"
all_paths = []
for operation in os.listdir(path_global):
    if os.path.isdir(join(path_global, operation)):
        for selem in os.listdir(join(path_global, operation)):
            all_paths += sorted(list_dir_joined(join(path_global, operation, selem)), key=lambda x: int(
                re.findall(r'version_(\d+)$', x)[0]
            ))[:-1]

In [4]:
reload_modules()
results = dr.DisplayResults().get_all_results_from_tensorboard(all_paths, load_long_args=False)

for r in results:
    r.update(r['args'])

Reading results: 100%|██████████| 126/126 [00:00<00:00, 160.19it/s]


In [5]:
df = pd.DataFrame(results)
df['dice'] = df['dice'].astype(float)
df['binary_mode_dice'] = df['binary_mode_dice'].astype(float)
df['operation'] = df['experiment_subname'].apply(lambda x: pathlib.Path(x).parent.stem)
df['selem'] = df['experiment_subname'].apply(lambda x: pathlib.Path(x).stem)

In [6]:
tb_path = df[
    (df['learning_rate'] == '0.01') &
    (df['loss_data_str'] == 'MaskedDiceLoss') &
    (df['init_weight_mode'] == 'CUSTOM_CONSTANT') &
    (df['bias_optim_mode'] == 'POSITIVE_INTERVAL_REPARAMETRIZED') &
    (df['operation'] == 'dilation') & 
    (df['selem'] == 'disk')
].tb_path
# with open(join(tb_path, 'seed.txt'), 'r') as f:
#     s = f.read()
# s

#### Binary Mode Success Number

In [8]:
changing = ["operation", "selem", ]
metrics = ['dice', 'binary_mode_dice']

df.groupby(changing)[metrics].mean()

dice  binary_mode_dice
operation    selem                             
black_tophat dcross  0.787829          0.584358
             disk    0.576229          0.469763
             hstick  0.800023          0.611364
closing      dcross  0.952607          0.928188
             disk    0.950629          0.887416
             hstick  0.982025          0.923033
dilation     dcross  0.998866          0.999181
             disk    0.992764          0.998750
             hstick  0.997077          1.000000
erosion      dcross  0.992340          0.990619
             disk    0.947644          0.980095
             hstick  0.992179          1.000000
opening      dcross  0.922787          0.850248
             disk    0.856041          0.695845
             hstick  0.966279          0.851179
white_tophat dcross  0.756640          0.591512
             disk    0.925504          0.541840
             hstick  0.761893          0.581860

In [12]:
print("Binary")
display(pd.DataFrame(df.loc[df['binary_mode_dice'] == 1, ["operation", "selem"]].value_counts().sort_values()).sort_values(by=['operation', 'selem']))

print("Real")
display(pd.DataFrame(df.loc[df['dice'] == 1, ["operation", "selem"]].value_counts().sort_values()).sort_values(by=['operation', 'selem']))


Binary


0
operation    selem    
black_tophat hstick  1
dilation     dcross  6
             disk    6
             hstick  7
erosion      dcross  6
             disk    6
             hstick  7

Real


0
operation    selem    
black_tophat disk    1
             hstick  2
closing      hstick  1
dilation     dcross  5
             disk    2
             hstick  4
erosion      dcross  5
             disk    2
             hstick  4
white_tophat disk    3

#### Best Set of hyperparameters

In [15]:
df.columns

Index(['args', 'tb_path', 'weights', 'normalized_weights', 'bias', 'dice',
       'baseline_dice', 'binary_mode_dice', 'convergence_dice', 'activation_P',
       'learned_selem', 'convergence_layer', 'target_selem',
       'target_operation', 'learned_weights_viz', 'learned_selems_viz',
       'stopping_reason', 'experiment_name', 'experiment_subname', 'name',
       'dataset_type', 'dataset_path', 'n_inputs', 'learning_rate',
       'batch_size', 'num_workers', 'freq_imgs', 'n_epochs', 'n_atoms',
       'atomic_element', 'kernel_size', 'init_weight_mode',
       'constant_activation_P', 'constant_weight_P', 'threshold_mode',
       'alpha_init', 'share_weights', 'loss', 'patience_loss',
       'patience_reduce_lr', 'closest_selem_method',
       'closest_selem_distance_fn', 'bias_optim_mode', 'loss_data_str',
       'initializer_method', 'initializer_args', 'bise_init_method',
       'optimizer', 'operations', 'loss_data', 'operation', 'selem'],
      dtype='object')

In [19]:
changing = [
    'loss_data_str',
    'learning_rate',
    'bise_init_method'
]
metrics = ['dice', 'binary_mode_dice']

results = {}
sums = {}
sums_style = {}


for L in range(1, len(changing) + 1):
    for subset in itertools.combinations(changing, L):
        df_summary = df.pivot_table(index=subset, columns=['operation', 'selem'], values='binary_mode_dice', aggfunc=lambda x: (x==1).any())#.astype(int)
        sums[subset] = pd.DataFrame(df_summary.sum(axis=1) / len(df_summary.columns))
        sums_style[subset] = sums[subset].style.format("{:,.0%}".format)
        results[subset] = df_summary




In [17]:
len(df_summary.columns) * len(df_summary)

126

In [21]:
print('Binary')
df_all = df.pivot_table(index=changing, columns=['operation', 'selem'], values=['binary_mode_dice'], aggfunc='mean').rename(columns={'dcross': 'dc', 'disk': 'di', 'hstick': 'hs'})
display(df_all.style.background_gradient(cmap='RdBu', vmin=0, vmax=1).format('{:.2f}'))

print('Real')
df_all = df.pivot_table(index=changing, columns=['operation', 'selem'], values=['dice'], aggfunc='mean').rename(columns={'dcross': 'dc', 'disk': 'di', 'hstick': 'hs'})
display(df_all.style.background_gradient(cmap='RdBu', vmin=0, vmax=1).format('{:.2f}'))

Binary


Real


In [31]:
df[
    (df['operation'] == "closing") & 
    (df['selem'] == "dcross") & 
    (df['bise_init_method'] == "CUSTOM_CONSTANT_RANDOM_BIAS") &
    (df['learning_rate'] == "0.01") &
    (df['loss_data_str'] == "MaskedMSELoss")
].tb_path.iloc[0]

'/hdd/aouadt/these/projets/3d_segm/deep_morpho/results/results_tensorboards/Bimonn_exp_63/multi/sybisel/softplus/diskorect/closing/dcross/version_3'

In [ ]:
def color_results(val):
    if val == True:
        return f'color: green'
    elif val == False:
        return f'color: red'
    elif val is None:
        return f'color: blue'

def display_color(df):
    display(df.style.applymap(color_results))

# results[tuple(changing)].style.applymap(color_results)
display_color(results[tuple(changing)])
# display_color(results[("loss_data_str", "init_weight_mode")])

In [ ]:
pd.DataFrame(results[tuple(changing)].sum(1) / len(results[tuple(changing)].columns)).style.format('{:.0%}')